<a href="https://www.kaggle.com/code/pedrobarrios/proyecto1-convolutional-kuzushijimnist?scriptVersionId=97746359" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [73]:
# Cargando librerias a usar
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import helper

print("[INFO] Librerias cargadas")

[INFO] Librerias cargadas


Carga de datos del dataset de Kuzushiji-MNIST de Anokas

In [74]:
#Funcion de apoyo para adaptar la forma del dataset
def resize(data):
    data = data.reshape(data.shape[0], 1, data.shape[1], data.shape[2])
    return data

print("[INFO] loading the KMNIST dataset...")
#Carga del dataset a arrays auxiliares
arr_train_img = np.load("../input/kuzushiji/kmnist-train-imgs.npz")['arr_0']
arr_train_label = np.load("../input/kuzushiji/kmnist-train-labels.npz")['arr_0']
arr_test_img = np.load("../input/kuzushiji/kmnist-test-imgs.npz")['arr_0']
arr_test_label = np.load("../input/kuzushiji/kmnist-test-labels.npz")['arr_0']

#Pasandolos a Tensor de pytorch
train_images = torch.Tensor(arr_train_img)/255
train_labels = torch.Tensor(arr_train_label).type(torch.LongTensor)
test_images = torch.Tensor(arr_test_img)/255
test_labels = torch.Tensor(arr_test_label).type(torch.LongTensor)

#Cambiando la forma del Tensor de las imagenes
train_images,test_images = resize(train_images),resize(test_images)

print("[INFO] KMNIST dataset 50% loaded")

[INFO] loading the KMNIST dataset...
[INFO] KMNIST dataset 50% loaded


In [75]:
#Verificando forma de la data importada y transformada
print("Imagenes")
print(train_images.shape)
print(test_images.shape)
print("Labels")
print(train_labels.shape)
print(test_labels.shape)

Imagenes
torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])
Labels
torch.Size([60000])
torch.Size([10000])


Clases, hiperparametros y Dataloader de pytorch estableciendo set de entrenamiento y set de pruebas

In [76]:
#HIPERPARAMETROS Y DEMAS

# Creando las clases para los diferentes familias de caracteres a clasificar
classes = ('O', 'Ki', 'Su', 'Tsu'
           , 'Na', 'Ha', 'Ma', 'Ya'
           , 'Re', 'Wo')

# Definimos el tamaños de los batch, los Epoch y el rate de aprendizaje
batch_size = 32
learning_rate = 0.01
epochs = 10
softmax_classes = 10

#Conversion de Tensor a Dataset de pytorch
trainset = torch.utils.data.TensorDataset(train_images,train_labels)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

testset = torch.utils.data.TensorDataset(test_images,test_labels)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

print("[INFO] KMNIST dataset loaded")

[INFO] KMNIST dataset loaded


MODELOS DE REDES NEURONALES

In [77]:
# Arquitectura V1
class ConvNN(nn.Module):
    def __init__(self, softmax_classes):
        super(ConvNN, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, softmax_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
                
        out = torch.flatten(x,1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out
    
print("[INFO] Arquitectura V1 utilizada")

[INFO] Arquitectura V1 utilizada


In [78]:
# Declaramos la red
model = ConvNN(softmax_classes)

# Definimos la metrica de la funcion de perdida
criterion = nn.CrossEntropyLoss()

# Seleccionamos nuestro optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loss=[]
tran_acc=[]

print("[INFO] Modelo, funcion de perdida y optimizador declarados")

[INFO] Modelo, funcion de perdida y optimizador declarados


In [79]:
#Entrenamiento
model.train()
for e in range(epochs):
    totalTrainLoss  = 0
    accuracy  = 0
    for (images, labels) in trainloader:
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        totalTrainLoss += loss.item()
        _,trainCorrect = torch.max(nn.functional.softmax(outputs, dim=1), 1)
        accuracy += (trainCorrect == labels).sum()
        
    else:
        totalTrainLoss = totalTrainLoss/len(trainloader)
        accuracy = (100 * accuracy.cpu().numpy()/len(train_images))
        
        print('Epoch:{} \t Training Loss:{:.6f} \t Accuracy:{}%'.format(
            e, 
            totalTrainLoss,
            accuracy
        ))
        
print("[INFO] Entrenamiento finalizado")

Epoch:0 	 Training Loss:0.370298 	 Accuracy:88.49%
Epoch:1 	 Training Loss:0.177100 	 Accuracy:94.595%
Epoch:2 	 Training Loss:0.122732 	 Accuracy:96.30333333333333%
Epoch:3 	 Training Loss:0.088016 	 Accuracy:97.36833333333334%
Epoch:4 	 Training Loss:0.062867 	 Accuracy:98.245%
[INFO] Entrenamiento finalizado


In [83]:
correctos = 0
todos = 0
model.eval()
with torch.no_grad():
    for images,labels in testloader:
        # apagamos los gradiantes para acelerar el proceso
        logps = model(images)
        # salida del resultado de las redes con sus prob 
        ps = torch.exp(logps)
        probab = list(ps.cpu().numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()
        if(true_label == pred_label):correctos += 1
        todos += 1



print('Images tested:{}\t Accuracy:{}%'.format( 
        todos,
        (correctos/todos)*100)
    )



Images tested:10000	 Accuracy:88.03999999999999%


In [ ]:
def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(classes, size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()